In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import copy
import os
path = os.getcwd()

In [3]:
from helper_functions.get_model_predictions import get_model_predictions
from helper_functions.get_BERT_vectors import get_BERT_vectors
from helper_functions.get_sentences import get_sentences
from helper_functions.get_similarity_matrix import get_similarity_matrix
from helper_functions.get_similar_sentences import get_similar_sentences 
from helper_functions.sample_new_sentences_per_original import sample_new_sentences_per_original

INFO:NferxLogger:URL: https://staging.nferx.com/api/is_authenticated, process-time: 0.3808112144470215


In [4]:
#read in the original set of sentences
df_old = pd.read_csv('starting_data/orig_data.csv')

In [5]:
#get the predictions using the current model
df_old = get_model_predictions(df_old,
                               'Drug_to_Target_20210519',
                               'model_01_060221_agree_2', 
                               'Classification',
                               'public')

https://pre-staging.nferx.com/tagrecorder/v1/getModelServingStatus
Using model server:  http://dev13.nferx.com:9001/predict


In [6]:
#keep just the cases where predicted label doesn't match manual label
#these are the sentences you want to boost 
df_old = df_old[df_old.label!=df_old.predicted_label]

In [7]:
#feed input into get sentences. use the same pairs from the old data
df_new = copy.deepcopy(df_old)
df_new = df_new[['entity1', 'entity2']].drop_duplicates()
df_new['max_count(default=1000)'] = [500] * len(df_new)
df_new['window(default=30)'] = [20] * len(df_new)
df_new['corpus(default=core)'] = [None] * len(df_new)
df_new['doc_token(default=None)'] = [None] * len(df_new)

failed, df_new = get_sentences(df_new)

print('{} inputs failed to get sentences'.format(len(failed)))

getting sentences for 56 inputs



fetched 21238 sentences in 44.41176247596741 seconds
0 inputs failed to get sentences


In [8]:
df_new = df_new.sample(n = 5000)

In [9]:
#get vectors for old and new sentences to be able to compare them 
old_vectors = get_BERT_vectors(df_old)
np.save('intermediate_data/old_vectors.npy', old_vectors)
new_vectors = get_BERT_vectors(df_new)
np.save('intermediate_data/new_vectors.npy', new_vectors)

In [10]:
#add the vectors as a column to the dataframes

new_vectors = np.load(path+ '/'+'intermediate_data/new_vectors.npy')
df_new['vec'] = list(new_vectors)
old_vectors = np.load(path+ '/'+'intermediate_data/old_vectors.npy')
df_old['vec'] = list(old_vectors)

In [11]:
# save the similarity matrix
# next step might require multiple re-runs with different parameters
# to get desired numbers
mat = get_similarity_matrix(df_old, df_new)
np.save('intermediate_data/sim_matrix.npy', mat)

In [12]:
mat = np.load(path+ '/'+'intermediate_data/sim_matrix.npy')
res = get_similar_sentences(df_new, mat, min_sim = 0.607, max_sim = 0.975, diff = 0.0015)

In [18]:
res = sample_new_sentences_per_original(res, 5)

In [19]:
res.to_csv(path+ '/'+'final_data/final.csv', index = False)
res[['sentence', 'entity1', 'entity2']].to_csv(path+ '/'+'final_data/final.tsv', index = False)